In [8]:
using JuMP

In [25]:
m = Model()
α_1 = 0.1399851
γ_1 = 0.464835
R = 6378*1000 #m 
η_A = 0.29
Q = 1367 #W/m^2
EN = 40
D_r = 5.3 #m
L = 9.777 
k = 1.38064852e-23 #J/K Boltzman constant R/Na
T_s = 135 #K
B = 8
N = 2000 # pixel width
η = 0.55
λ_v = 500e-9 #nm
f = 2.2e9 #Hz
c = 3e8  #m/s
λ_c = c/f
G_R = η*π*(D_r/λ_c)^2
ρ_A = 10 #kg/m^2
ρ_p = 100 #kg/m^1.5
ρ_T = 2 #kg/m^1.5
ρ_P = 500 #kg*km
ρ_b = 0.001 #kg/J
P_l = 5 #W
μ = 3.986005e14 # m^3/s^2 standard gravitational parameter
X_r = 20 #m/px
m_c = 0.2 #kg, some required weight
η_S = 0.2 # 20% of total mass is structural mass

# Subsystem specific parameters
@variable(m, D_p)  # payload optical aperture diameter
@variable(m, D_T)  # transmitter antenna diameter
@variable(m, A)    # solar panel surface area

# Mass related parameters
@variable(m, m_t)  # total mass
@variable(m, m_b)  # battery mass
@variable(m, m_A)  # solar panel mass
@variable(m, m_p)  # payload mass
@variable(m, m_T)  # transmitter mass
@variable(m, m_S)  # structural mass
@variable(m, m_P)  # propulsion mass

# Power related parameters
@variable(m, P_t) # total power
@variable(m, P_T) # transmitter power
@variable(m, E_b) # battery energy

# Orbit related parameters
@variable(m, h)   # altitude
@variable(m, a)   # semi-major axis
@variable(m, T)   # orbit period 
@variable(m, r)   # worst case communications distance
@variable(m, d)   # daylight fraction of orbit
@variable(m, e)   # eclipse faction of orbit
@variable(m, g)   # ground station viewing fraction of orbit

# I know there is a cleaner way to do this instead of declaring @NLconstraint 
# every time but here we go

# Power and communications
@NLconstraint(m, P_t == A*η_A*Q) # all power generated by solar panel
@constraint(m, P_T + P_l <= d*P_t)
@NLconstraint(m, E_b >= P_t*e*T)
@NLconstraint(m, EN == P_T*G_R*X_r*g*T*η*D_T^2/(L*k*T_s*R*2*π*N*B*4*r^2))

# Payload performance
@NLconstraint(m, X_r == h*λ_v/D_p)

# Orbit
@constraint(m, e >= 1 - d)
@constraint(m, d <= g + 0.5)
@NLconstraint(m, a >= R+h)  # this is technically an equality, 
                            # but dont know if you allow signomial equalities
@NLconstraint(m, g == α_1*(h/R)^γ_1)
@NLconstraint(m, T == 2*π*sqrt(a^3/μ))
@NLconstraint(m, r^2 >= h^2 + 2*R*h) # this is also an equality technically

# Mass budgets
@NLconstraint(m, m_b == ρ_b*E_b)
@NLconstraint(m, m_p == ρ_p*D_p^1.5)
@NLconstraint(m, m_A == ρ_A*A)
@NLconstraint(m, m_T == ρ_T*D_T^1.5)
@NLconstraint(m, m_P == ρ_P/h)
@NLconstraint(m, m_S == η_S*m_t)
@constraint(m, m_b+m_p+m_A+m_T+m_P+m_S<= m_t)

m_b + m_p + m_A + m_T + m_P + m_S - m_t <= 0

In [26]:
m

Feasibility problem with:
 * 3 linear constraints
 * 1 quadratic constraint
 * 14 nonlinear constraints
 * 20 variables
Solver is default solver